凸编程规则(DCP)是一种用于从给定的基函数库构造具有已知曲率的数学表达式的系统。CVXPY使用DCP来确保指定的优化问题是凸的。

本教程的这一部分解释了DCP的规则以及它们在CVXPY中的应用。

请访问[网址](http://dcp.stanford.edu)以获取更多交互式DCP介绍。

# `Expressions`

CVXPY中的表达式由变量，参数，数值常量（如Python浮点数和Numpy矩阵），标准算术运算符`+`， `-` ，`*`，`/`和一个函数库组成。 以下是CVXPY表达式的一些示例：

In [1]:
import cvxpy as cvx

# Create variables and parameters.
x, y = cvx.Variable(), cvx.Variable()
a, b = cvx.Parameter(), cvx.Parameter()

# Examples of CVXPY expressions.
3.69 + b/3
x - 4*a
cvx.sqrt(x) - cvx.minimum(y, x - a)
cvx.maximum(2.66 - cvx.sqrt(y), cvx.square(x + 2*y))

Expression(CONVEX, NONNEGATIVE, ())

表达式可以是标量，向量或矩阵。 表达式的维度存储为`expr.shape`。条目总数由`expr.size`给出，而维数由`expr.ndim`给出。如果表达式的使用方式在给定维度时没有意义，例如添加不同大小的矩阵，则CVXPY将引发异常。shapes行为在算术运算下的语义与NumPy ndarrays相同（除了某些广播被禁止）。

In [2]:
import numpy

X = cvx.Variable((5, 4))
A = numpy.ones((3, 5))

# Use expr.shape to get the dimensions.
print(("dimensions of X:", X.shape))
print(("size of X:", X.size))
print(("number of dimensions:", X.ndim))
print(("dimensions of sum(X):", cvx.sum(X).shape))
print(("dimensions of A*X:", (A*X).shape))

# ValueError raised for invalid dimensions.
try:
    A + X
except ValueError as e:
    print(e)

('dimensions of X:', (5, 4))
('size of X:', 20)
('number of dimensions:', 2)
('dimensions of sum(X):', ())
('dimensions of A*X:', (3, 4))
Cannot broadcast dimensions  (3, 5) (5, 4)


CVXPY使用DCP分析来确定每个表达式的符号和曲率。

# `Sign`

每个(子)表达式被标记为正(非负)，负(非正)，零或未知。

较大表达式的符号由其子表达的符号确定。例如，表达式`expr1 * expr2`的符号是：
+ `Zero` 如果任一表达式的符号为零
+ `Positive` 如果expr1和expr2具有相同（已知）的符号
+ `Negative` 如果expr1和expr2有相反的（已知）符号
+ `Unknown` 如果任一表达式具有未知符号

给定表达式的符号总是正确的。但是当通过更复杂的分析计算出符号时，DCP符号分析可能将表达标记为未知符号。例如，根据上述规则，`x * x`符号为`Positive`，但DCP分析为`Unknown`符号。

CVXPY通过查看它们的值来确定常量的符号。对于标量常量，这很简单。所有正（负）条目的矢量和矩阵常数标记为正（负）。同时有正负项的向量和矩阵常量标记为未知符号。

表达式的符号存储在`expr.sign`：

In [3]:
x = cvx.Variable()
a = cvx.Parameter(nonpos=True)
c = numpy.array([1, -1])

print("sign of x:", x.sign)
print("sign of a:", a.sign)
print("sign of square(x):", cvx.square(x).sign)
print("sign of c*a:", (c*a).sign)

sign of x: UNKNOWN
sign of a: NONPOSITIVE
sign of square(x): NONNEGATIVE
sign of c*a: UNKNOWN


# `Curvature`

每个（子）表达式被标记为以下曲线之一（就其变量而言）:

![curvature](curvature.png)

使用下面给出的曲率规则。与符号分析一样，结论总是正确的，但简单的分析可以将表达标记为未知，即使它们是凸的或凹的。请注意，任何常量表达式都是仿射的，并且任何仿射表达式都是凸凹的。

## `Curvature`规则

DCP分析基于将凸分析应用于每个（子）表达式的通用组合定理。

1. $f(expr_1, expr_2, ..., expr_n)$是凸的， 如果$f$是一个凸函数，并且对于每个$expr_i$，下列条件之一成立：
 + $f$随参数$i$递增且$expr_i$是凸的
 + $f$随参数$i$递减且$expr_i$是凹的
 + $expr_i$是仿射或常量

2. $f(expr_1, expr_2, ..., expr_n)$是凹的， 如果$f$是一个凹函数，并且对于每个$expr_i$，下列条件之一成立：
 + $f$随参数$i$递增且$expr_i$是凹的
 + $f$随参数$i$递减且$expr_i$是凸的
 + $expr_i$是仿射或常量

3. $f(expr_1, expr_2, ..., expr_n)$是仿射，如果$f$是仿射且每个$expr_i$都是仿射。

如果以上三个规则都不适用，则表达式$f(expr_1, expr_2, ..., expr_n)$被标记为未知曲率。

函数随参数变化是增加还是减少可能取决于参数的符号。例如，对于正参数平方增加，而对于负数平方减少。

表达式的曲率存储为`expr.curvature`：

In [4]:
x = cvx.Variable()
a = cvx.Parameter(nonneg=True)

print("curvature of x:", x.curvature)
print("curvature of a:", a.curvature)
print("curvature of square(x):", cvx.square(x).curvature)
print("curvature of sqrt(x):", cvx.sqrt(x).curvature)

curvature of x: AFFINE
curvature of a: CONSTANT
curvature of square(x): CONVEX
curvature of sqrt(x): CONCAVE


# 中缀操作

中缀运算符`+`，`-` ，`*`，`/`的处理方式与函数完全相同。中缀运算符`+`和`-`是仿射的，所以上面的规则适用于标记曲率。例如，如果`expr1`和`expr2`是凸的，`expr1 + expr2`被标记为凸。

只有当其中一个表达式为常量时，才允许`expr1 * expr2`。如果两个表达式都是非常量，则CVXPY将引发异常。`expr1 / expr2`仅当`expr2`是标量常量时才被允许。上面的曲率规则同样适用。例如，当`expr1`是凹的，`expr2`是负常数时，`expr1 / expr2`是凸的。

## 例1

DCP分析将表达式分解为子表达式。下面的树形视图显示了这个表达式如何用于表达式`2 * square(x）+ 3`。每个子表达式都显示在一个蓝色框中。 我们将其曲率标记在左侧，其符号标记在右侧。

![例子1](example1.png)

## 例2

将DCP规则应用到`sqrt(1 + square(x))`表达式。

![例子2](example2.png)

变量`x`具有仿射曲率和未知符号。平方函数是凸的，对于未知符号的参数是非单调的。它可以将仿射表达式`x`作为参数; 结果`square(x)`是凸的。

算术运算符`+`是仿射和增加的，所以对于凸函数，由曲率规则，组合`1 + square(x)`是凸的。函数`sqrt`是凹的并且增加，这意味着它只能采用凹面参数。由于`1 + square(x)`是凸的，因此`sqrt(1 + square(x))`违反DCP规则，不能验证为凸。

实际上，`sqrt(1 + square(x))`是x的凸函数，但是DCP规则不能验证凸性。如果表达式被写为`norm(vstack(1, x), 2)`，则该向量`[1，x]`的L2范数与`sqrt(1 + square(x))`具有相同的值，那么它将使用DCP规则认证为凸。

In [5]:
print("sqrt(1 + square(x)) curvature:",
      cvx.sqrt(1 + cvx.square(x)).curvature)
print("norm(hstack([1, x]), 2) curvature:",
      cvx.norm(cvx.hstack([1, x]), 2).curvature)

sqrt(1 + square(x)) curvature: UNKNOWN
norm(hstack([1, x]), 2) curvature: CONVEX


# DCP问题

一个问题由一个目标和一系列约束构成的。如果问题遵循DCP规则，那么CVXPY保证是凸的且可解。DCP规则要求问题目标有以下两种形式之一：
+ Minimize(凸)
+ Maximize(凹)

DCP规则下唯一有效的限制是：
+ affine == affine
+ convex <= concave
+ concave >= convex

您可以通过调用`object.is_dcp()`来检查问题，约束或目标是否满足DCP规则。以下是DCP和非DCP问题的一些示例：

In [6]:
x = cvx.Variable()
y = cvx.Variable()

# DCP problems.
prob1 = cvx.Problem(cvx.Minimize(cvx.square(x - y)), [x + y >= 0])
prob2 = cvx.Problem(
    cvx.Maximize(cvx.sqrt(x - y)),
    [2 * x - 3 == y, cvx.square(x) <= 2])

print(("prob1 is DCP:", prob1.is_dcp()))
print(("prob2 is DCP:", prob2.is_dcp()))

# Non-DCP problems.

# A non-DCP objective.
obj = cvx.Maximize(cvx.square(x))
prob3 = cvx.Problem(obj)

print(("prob3 is DCP:", prob3.is_dcp()))
print(("Maximize(square(x)) is DCP:", obj.is_dcp()))

# A non-DCP constraint.
prob4 = cvx.Problem(cvx.Minimize(cvx.square(x)), [cvx.sqrt(x) <= 2])

print("prob4 is DCP:", prob4.is_dcp())
print("sqrt(x) <= 2 is DCP:", (cvx.sqrt(x) <= 2).is_dcp())

('prob1 is DCP:', True)
('prob2 is DCP:', True)
('prob3 is DCP:', False)
('Maximize(square(x)) is DCP:', False)
prob4 is DCP: False
sqrt(x) <= 2 is DCP: False


如果在非DCP问题上调用p`problem.solve()`，则CVXPY将引发异常。

In [7]:
# A non-DCP problem.
prob = cvx.Problem(cvx.Minimize(cvx.sqrt(x)))

try:
    prob.solve()
except Exception as e:
    print(e)

Problem does not follow DCP rules.
